In [1]:
from langchain_openai import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [2]:
OPENAI_API_KEY = '#'
model = 'gpt-4o-mini'

In [3]:
# {
#   "type": "service_account",
#   "project_id": "#",
#   "private_key_id": "#4",
#   "private_key": "#\n",
#   "client_email": "#",
#   "client_id": "#",
#   "auth_uri": "https://accounts.google.com/o/oauth2/auth",
#   "token_uri": "https://oauth2.googleapis.com/token",
#   "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
#   "client_x509_cert_url": "#"
# }

In [8]:
service_account_file = "service-account-key.json" # Change to where your service account key file is located

project = "##"
dataset = "dt_maven_sample_datasets"
table = "sample_data_marketing_dnk"

sqlalchemy_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'

In [9]:
from sqlalchemy import create_engine

In [10]:
engine = create_engine(sqlalchemy_url)

In [11]:
db = SQLDatabase(engine=engine,
                  include_tables=[table])

In [12]:
# Initialize LLM
llm = ChatOpenAI(model=model, temperature=0, openai_api_key=OPENAI_API_KEY)

In [13]:
# db = SQLDatabase.from_uri(sqlalchemy_url)

In [14]:
db_chain = SQLDatabaseChain.from_llm(llm,
                                      db=db,
                                      verbose=True,
                                      use_query_checker=True,
                                      top_k=100,
                                      return_sql=True
                                      )

In [15]:
out = db_chain.invoke("what's the schema of this table?")



> Entering new SQLDatabaseChain chain...
what's the schema of this table?
SQLQuery:
> Finished chain.


In [16]:
out.keys()

dict_keys(['query', 'result'])

In [21]:
out['result']

"SQLQuery: SELECT column_name, data_type \nFROM `sample_data_marketing_dnk`.INFORMATION_SCHEMA.COLUMNS \nWHERE table_name = 'sample_data_marketing_dnk' \nORDER BY ordinal_position;"

In [22]:
db.run("""SELECT column_name, data_type
FROM `datachannel-238509.dt_maven_sample_datasets.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'sample_data_marketing_dnk'
ORDER BY ordinal_position;""")

"[('f__platform', 'STRING'), ('f__channel', 'STRING'), ('f__breakdown', 'STRING'), ('country_code', 'STRING'), ('region', 'STRING'), ('date', 'DATE'), ('refresh_ts', 'TIMESTAMP'), ('ad_group_id', 'STRING'), ('ad_group_name', 'STRING'), ('ad_id', 'STRING'), ('ad_name', 'STRING'), ('campaignid', 'STRING'), ('campaignname', 'STRING'), ('sku', 'STRING'), ('userlist_access', 'STRING'), ('product_name', 'STRING'), ('brand', 'STRING'), ('reporting_category', 'STRING'), ('reporting_subcategory', 'STRING'), ('internal_name', 'STRING'), ('internal_sku', 'STRING'), ('product_group_1', 'STRING'), ('product_group_2', 'STRING'), ('product_group_3', 'STRING'), ('product_group_4', 'STRING'), ('product_group_5', 'STRING'), ('spend', 'NUMERIC'), ('impressions', 'NUMERIC'), ('clicks', 'INT64'), ('conversions', 'INT64'), ('revenue', 'NUMERIC')]"

: 

In [16]:
# toolkit = SQLDatabaseToolkit(db=db, llm=llm)
# agent_executor = create_sql_agent(llm=llm,
#                                     toolkit=toolkit,
#                                     verbose=True,
#                                     top_k=1000,
#                                     )